# Part 2 - Querying the Data
Construct functionality that allows you to query the data. If we search for a term, like “music,” we would like to know some subset of the following:
* How many tweets were posted containing the term on each day?
* How many unique users posted a tweet containing the term?
* How many likes did tweets containing the term get, on average?
* Where (in terms of place IDs) did the tweets come from?
* What times of day were the tweets posted at? 
* Which user posted the most tweets containing the term?

## Get Connection and reference to collection

In [9]:
import pymongo
from pymongo import MongoClient
from pymongo.collection import Collection
import urllib.parse
import os
import pandas as pd
import time

In [4]:
def get_client() -> MongoClient:
    """
        creates a client for connecting to the database and returns it.
    """
    client = MongoClient('mongodb',
                     username=os.environ['MONGO_INITDB_ROOT_USERNAME'],
                     password=os.environ['MONGO_INITDB_ROOT_PASSWORD'],
                     authMechanism='SCRAM-SHA-256')

    try:
        # The ismaster command is cheap and does not require auth.
        client.admin.command('ismaster')
        print("MongoDB server Connection Test successful")
    except ConnectionFailure:
        print("MongoDB server not available")
    
    return client

In [5]:
client = get_client()
db = client['nio']
tweets_col = db['tweets']

MongoDB server Connection Test successful


## First Step: Getting Tweets
I wanted to start by writing a function that would just generally get all the tweets containing the term. Each of the questions above can be answered with just this subset of data. The `get_tweets(term)` function accepts a `term` as a string and returns a pandas `Dataframe` representing all of the records containing this term in the `text` field. 

While tha solution for each question still requires an additional step or two to transform and filter the day, these are all performed in python by the API server in order to keep the SQL queries simple. This might not be ideal depending on the needs of the project, but this decision allowed me to complete more of the requirements for this project.

In [10]:
def get_tweets(term: str, tweets_col: Collection) -> pd.DataFrame:
    """queries the database for records containg a string matching `term`

    Parameters
    ----------
    term: str, required
        The term to query the database with
        
    """
    q = {"$text":
         {"$search": term},
    }

    print("retreiving...")
    start_t = time.time()
    results = list(tweets_col.find(q, {"text":1, "ts1":1, "place_id": 1, "author_id":1, "author_handle":1, "like_count":1}))
    print(f"Query took {time.time()-start_t} seconds.")

    print("counting...")
    results_df = pd.DataFrame(results).set_index('_id')
    results_df['datetime'] = pd.to_datetime(results_df['ts1'])
    results_df['like_count'] = pd.to_numeric(results_df['like_count'], downcast='integer', errors='coerce')

    return results_df

In [11]:
term_df = get_tweets("music", tweets_col)

retreiving...
Query took 0.721980094909668 seconds.
counting...


In [12]:
term_df.head()

,ts1,text,author_id,author_handle,like_count,datetime
_id,,,,,,
6681b24534f58518da1ebe51,2022-03-01 07:00:10.314927-05:00,If I hate anything...I hate a music snob. I li...,52985792,DaCaptivating1,4,2022-03-01 07:00:10.314927-05:00
6681b25334f58518da286eca,2022-03-01 09:26:23.141990-05:00,anyone pls explain 2 me y middle eastern &amp;...,363800896,SwallowBricks,93,2022-03-01 09:26:23.141990-05:00
6681b24834f58518da200060,2022-03-01 07:17:55.934769-05:00,"@Michigrimk Oh, and if it's about music, it's ...",1070440946544840705,TheyMakeMad,1,2022-03-01 07:17:55.934769-05:00
6681b25134f58518da276210,2022-03-01 08:58:29.955943-05:00,@dramaticgorl @_lexadyanni This music you're t...,869062143685120001,immaputdishere,0,2022-03-01 08:58:29.955943-05:00
6681b24534f58518da1e7009,2022-03-01 09:19:52.324385-05:00,Guys it’s not about finding non-copyrighted mu...,713087063252418560,NintyPrime,6,2022-03-01 09:19:52.324385-05:00


### How many tweets were posted containing the term on each day?

In [13]:
def tweet_count_per_day(term_df):
    counts_df = term_df.groupby([term_df['datetime'].dt.date]).count()

    data = []
    
    for ind, row in counts_df.iterrows():
        record = {'date':ind.strftime("%Y-%m-%d"),
                  'count':int(row['text'])}
        data.append(record)

    return data

In [14]:
tweet_count_per_day(term_df)

[{'date': '2022-01-04', 'count': 276},
 {'date': '2022-01-05', 'count': 1219},
 {'date': '2022-01-22', 'count': 14},
 {'date': '2022-03-01', 'count': 24513}]

### How many unique users posted a tweet containing the term?

In [15]:
author_vc = term_df['author_id'].value_counts()
author_vc.shape

(18532,)

### How many likes did tweets containing the term get, on average?

In [16]:
term_df['like_count'].mean()

112.73311044500807

### Where (in terms of place IDs) did the tweets come from?

In [17]:
test_df = term_df

if 'place_id' not in test_df.columns:
    place_ids = []
else:
    place_ids = list(test_df['place_id'].unique())

print(len(place_ids))

0


### What times of day were the tweets posted at?
Since the number of unique times of day is likely very large, I bucketed the times by "morning", "afternoon", "evning", and "overnight" using my own judgement for the boundaries of each.

In [18]:
morning_df = term_df.set_index('datetime').between_time('05:00:00', '12:00:00')
afternoon_df = term_df.set_index('datetime').between_time('12:00:00','17:00:00')
evening_df = term_df.set_index('datetime').between_time('17:00:00','23:59:59')
overnight_df = term_df.set_index('datetime').between_time('00:00:00','05:00:00')

### Which user posted the most tweets containing the term?

In [19]:
term_df['author_handle'].value_counts().index[0]

'RadDance'

# Putting it into a method
I interpreted the requirements of the assignment to mean that all of these questions could be answered easily by a single response payload from the API server. For this reason the following function was designed to take in a term and produce a json like in the following example:

```
{'term': 'music', # the term that was queried
 'time_to_complete_query': 0.655991792678833, # The time between the api servers query request and receiving the response.
 'counts_by_day': # each day the term appeared in a tweet and the number of times
    [
        {'date': '2022-01-04', 'count': 276},
        {'date': '2022-01-05', 'count': 1219},
        {'date': '2022-01-22', 'count': 14},
        {'date': '2022-03-01', 'count': 24513}
    ],
 'users': 18532, # the number of unique users who tweeted the term
 'avg_likes_per_tweet': 112, # the average like for posts that used the term
 'place_ids': [], # the unique place ids of tweets that used the term
 'times_of_day': # the number of times the term was used in a tweet posted at this time of day
    {'morning': 25732,
     'afternoon': 9,
     'evening': 281,
     'overnight': 0}
    }
}
```

In [20]:
def query_term(term, tweets_col):
    start_t = time.time()
    term_df = get_tweets(term, tweets_col)
    time_to_query = time.time() - start_t
    # How many tweets were posted containing the term on each day?
    daily_counts = tweet_count_per_day(term_df)

    # How many unique users posted a tweet containing the term?
    author_vc = term_df['author_id'].value_counts()
    unique_users = len(author_vc)

    # How many likes did tweets containing the term get, on average?
    avg_likes_per_tweet = term_df['like_count'].mean()

    # Where (in terms of place IDs) did the tweets come from?
    place_ids = []
    if 'place_id' in term_df.columns:
        place_ids = list(term_df['place_id'].unique())

    # What times of day were the tweets posted at?
    term_df = term_df.set_index('datetime')
    morning_df = term_df.between_time('05:00:00', '12:00:00')
    afternoon_df = term_df.between_time('12:00:00','17:00:00')
    evening_df = term_df.between_time('17:00:00','23:59:59')
    overnight_df = term_df.between_time('00:00:00','05:00:00')

    times_of_day = {'morning':len(morning_df),
                    'afternoon':len(afternoon_df),
                    'evening':len(evening_df),
                    'overnight':len(overnight_df)}

    most_used_by = term_df['author_handle'].value_counts().index[0]
    
    results = {'term':term,
               'time_to_complete_query':time_to_query,
               'counts_by_day':daily_counts,
               'users':unique_users,
               'avg_likes_per_tweet':int(avg_likes_per_tweet),
               'place_ids':place_ids,
               'times_of_day':times_of_day,
               'most_used_by':most_used_by}
    return results

In [21]:
result = query_term("music", tweets_col)
result

retreiving...
Query took 0.15152907371520996 seconds.
counting...


{'term': 'music',
 'time_to_complete_query': 0.3309319019317627,
 'counts_by_day': [{'date': '2022-01-04', 'count': 276},
  {'date': '2022-01-05', 'count': 1219},
  {'date': '2022-01-22', 'count': 14},
  {'date': '2022-03-01', 'count': 24513}],
 'users': 18532,
 'avg_likes_per_tweet': 112,
 'place_ids': [],
 'times_of_day': {'morning': 25732,
  'afternoon': 9,
  'evening': 281,
  'overnight': 0},
 'most_used_by': 'RadDance'}

# Check that it is json serializable

In [12]:
import json

In [19]:
json.dumps(result)

'{"term": "music", "time_to_complete_query": 0.6354808807373047, "counts_by_day": [{"date": "2022-01-04", "count": 276}, {"date": "2022-01-05", "count": 1219}, {"date": "2022-01-22", "count": 14}, {"date": "2022-03-01", "count": 24513}], "users": 18532, "avg_likes_per_tweet": 112.733, "place_ids": [], "times_of_day": {"morning": 25732, "afternoon": 9, "evening": 281, "overnight": 0}}'

# Next Steps
After working out these functions in a jupyter notebook, I copied them into `/api_server/db.py` to be used in the api server code.